In [1]:
include("../src/AlgebraicIdeals.jl")

Singular.jl, based on
                     SINGULAR                                 /  
 A Computer Algebra System for Polynomial Computations       /  Singular.jl: 0.4.1 
                                                           0<   Singular   : 2.3.1-4
 by: W. Decker, G.-M. Greuel, G. Pfister, H. Schoenemann     \   
FB Mathematik der Universitaet, D-67653 Kaiserslautern        \
     


from_singular (generic function with 1 method)

## Predator-pray model

The predator prey-model is the following classical ODE system (you can think of $x$ being the number of rabbits and $y$ being the number of wolves)

$\begin{cases}
  x' = ax - bxy,\\
  y' = -cy + dxy
\end{cases}$

Assuming we cannot count the wolves but only the rabbits, we would like to obtain an equation in $x$ only.

First we create the coefficient field $\mathbb{Q}(a, b, c, d)$

In [2]:
coef_ring, (a, b, c, dd) = Nemo.PolynomialRing(Nemo.QQ, ["a", "b", "c", "d"])
F = FractionField(coef_ring)

Fraction field of Multivariate Polynomial Ring in a, b, c, d over Rational Field

And then the ring of differential polynomials together with the system

In [3]:
R, (x, y) = DifferentialPolynomialRing(F, ["x", "y"])
a, b, c, dd = map(x -> R(x), [a, b, c, dd])

eqs = [
    d(x) - a * x + b * x * y,
    d(y) + c * y - dd * x * y
]

2-element Array{DiffPoly,1}:
 b*x^(0)*y^(0) - a*x^(0) + x^(1)
 -d*x^(0)*y^(0) + c*y^(0) + y^(1)

In order to preform elimination, we add a derivative of each of the equations:

In [4]:
for i in 1:2
    push!(eqs, d(eqs[i]))
end
eqs

4-element Array{DiffPoly,1}:
 b*x^(0)*y^(0) - a*x^(0) + x^(1)
 -d*x^(0)*y^(0) + c*y^(0) + y^(1)
 b*x^(0)*y^(1) + b*x^(1)*y^(0) - a*x^(1) + x^(2)
 -d*x^(0)*y^(1) - d*x^(1)*y^(0) + c*y^(1) + y^(2)

Now we convert them to normal polynomial with `lex` ordering so that $y$'s are higher than $x$'s.

In [6]:
Rsing, pls = get_singular_polys(eqs, :lex, lex_ranking(["y", "x"]))

(Singular Polynomial Ring (Coeffs(17)),(y^(0),y^(1),y^(2),x^(0),x^(1),x^(2)),(lp(6),C), spoly{Singular.n_unknown{AbstractAlgebra.Generic.Frac{Nemo.fmpq_mpoly}}}[b*y^(0)*x^(0)+-a*x^(0)+x^(1), -d*y^(0)*x^(0)+c*y^(0)+y^(1), b*y^(0)*x^(1)+b*y^(1)*x^(0)+-a*x^(1)+x^(2), -d*y^(0)*x^(1)+-d*y^(1)*x^(0)+c*y^(1)+y^(2)])

... and compute the Groebner basis

In [8]:
gb = gens(Singular.std(Ideal(Rsing, pls)))

5-element Array{spoly{Singular.n_unknown{AbstractAlgebra.Generic.Frac{Nemo.fmpq_mpoly}}},1}:
 (1)*x^(0)^3 + ((-1)//a)*x^(0)^2*x^(1) + ((-c)//d)*x^(0)^2 + (c//(a*d))*x^(0)*x^(1) + (1//(a*d))*x^(0)*x^(2) + ((-1)//(a*d))*x^(1)^2
 (1)*y^(2)*x^(1) + ((a*c^2*d)//b)*x^(0)^2 + ((a*c*d - c^2*d)//b)*x^(0)*x^(1) + ((-a*c^3)//b)*x^(0) + ((-a*d - c*d)//b)*x^(1)^2 + (d//b)*x^(1)*x^(2) + ((-a*c^2 + c^3)//b)*x^(1) + (c^2//b)*x^(2)
 (1)*y^(2)*x^(0) + ((-1)//c)*y^(2)*x^(1) + ((-2*a*d)//b)*x^(0)*x^(1) + (d//b)*x^(0)*x^(2) + ((a*d + c*d)//(b*c))*x^(1)^2 + ((-d)//(b*c))*x^(1)*x^(2) + ((a*c)//b)*x^(1) + ((-c)//b)*x^(2)
 (1)*y^(1) + (1//c)*y^(2) + ((-a*d)//(b*c))*x^(1) + (d//(b*c))*x^(2)
 (1)*y^(0) + (1//c)*y^(1) + ((-a*d)//(b*c))*x^(0) + (d//(b*c))*x^(1)

Note that the first element involves $x$'s only. We convert it back to the differential ring and muliply by $ad$ for convenience.

In [9]:
ioequation = a * dd * from_singular(gb[1], R)

a*d*x^(0)^3 - d*x^(0)^2*x^(1) - a*c*x^(0)^2 + c*x^(0)*x^(1) + x^(0)*x^(2) - x^(1)^2

Then we conclude that the functions of the parameters that can be identified by a series of experiments are

$\mathbb{C}(ad, d, ac, c) = \mathbb{C}(a, c, d)$

## Squaring the Painleve I function

Painleve transcendents are the function satisfying a differential equation of the form $y'' = f(t, y, y')$ of degree two and not expressable in terms of "simpler" functions.

The simplest one is the first Painleve equation:

$y'' = 6y^2 + t$

We would like to derive a differential equation satisfied by $z = y^2$.

In [10]:
R, (t, y, z) = DifferentialPolynomialRing(Nemo.QQ, ["t", "y", "z"])

(Differential polynomial ring over Rational Field in t, y, z, (t^(0), y^(0), z^(0)))

In [11]:
eqs = [
    d(t) - 1,
    d(y, 2) - 6 * y^2 - t,
    z - y^2
]

3-element Array{DiffPoly,1}:
 t^(1) - 1
 -t^(0) - 6*y^(0)^2 + y^(2)
 -y^(0)^2 + z^(0)

In [12]:
ord = 3
for i in 1:length(eqs)
    for j in 1:ord
        push!(eqs, d(eqs[i], j))
    end
end

In [13]:
Rsing, pls = get_singular_polys(eqs);

In [14]:
gb = gens(Singular.std(Ideal(Rsing, pls)));
p = from_singular(gb[1], R)

z^(0)^5*z^(1)^2 + 1//3*z^(0)^5*z^(1) + 1//36*z^(0)^5 - 1//144*z^(0)^4*z^(3)^2 + 1//48*z^(0)^3*z^(1)*z^(2)*z^(3) - 1//96*z^(0)^2*z^(1)^3*z^(3) - 1//64*z^(0)^2*z^(1)^2*z^(2)^2 + 1//64*z^(0)*z^(1)^4*z^(2) - 1//256*z^(1)^6

## Challenge

How can we get a differential equation satisfied by $\frac{1}{\sin t} + \frac{1}{\cos t}$?